In [1]:
import sys
!conda install --yes --prefix {sys.prefix} requests
!conda install --yes --prefix {sys.prefix} bs4

import requests #package for http requests
import bs4 # package for html parsing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
current = 2021
years = list(range(current, current - 72, -1))

In [3]:
race_results = pd.DataFrame()
for year in years:
    link = 'https://www.formula1.com/en/results.html/'+str(year)+'/races.html'
    source = requests.get(link)
    soup = bs4.BeautifulSoup(source.text,'lxml')
    table = soup.find_all('table')[0] 
    df = pd.read_html(str(table), flavor='bs4', header=[0])[0]
    df['Year'] = year
    race_results = race_results.append(df,ignore_index=True)
race_results = race_results[['Grand Prix','Date','Winner','Car','Laps','Time','Year']]

In [4]:
driver_results = pd.DataFrame()
for year in years:
    dlink = 'https://www.formula1.com/en/results.html/'+str(year)+'/drivers.html'
    dsource = requests.get(dlink)
    dsoup = bs4.BeautifulSoup(dsource.text,'lxml')
    dtable = dsoup.find_all('table')[0] 
    ddf = pd.read_html(str(dtable), flavor='bs4', header=[0])[0]
    ddf['Year'] = year
    driver_results = driver_results.append(ddf,ignore_index=True)
driver_results = driver_results[['Pos','Driver','Nationality','Car','PTS','Year']]

In [5]:
driver_salaries = pd.DataFrame()
for year in ['2021','2020','2019','2018']:

    slink = 'https://www.spotrac.com/formula1/'+str(year)+'/'
    ssource = requests.get(slink)
    ssoup = bs4.BeautifulSoup(ssource.text,'lxml')
    rows = ssoup.find_all('tr') 
    drivers =[]
    for row in range(len(rows)):
        drivers_row = []
        columns = rows[row].findAll('td')
        for column in columns:
            drivers_row.append(column.getText())
        drivers.append(drivers_row)
    salaries = pd.DataFrame(drivers).dropna()
    salaries['Year']=year
    driver_salaries = driver_salaries.append(salaries,ignore_index=True)   
    
new_colnames = ['Driver','Age','Country','Salary','Year']
driver_salaries.columns = new_colnames
driver_salaries['Driver'] = [row.strip() for row in driver_salaries['Driver']]

In [6]:
driver_results['Driver'] = driver_results['Driver'].str.rsplit(' ',1).str[0]
driver_results['Driver'] = driver_results['Driver'].str.replace('  ', ' ')
driver_results['Driver'] = driver_results['Driver'].str.replace('   ', ' ')
race_results['Winner'] = race_results['Winner'].str.rsplit(' ',1).str[0]